In [ ]:
import polars   as pl
import networkx as nx
pl.Config(tbl_width_chars=500)
pl.Config(tbl_cols=-1)
import rtsvg
rt = rtsvg.RACETrack()
from  polars_spring_layout    import PolarsSpringLayout
from  linknode_graph_patterns import LinkNodeGraphPatterns
import random
import copy
import time
'''
edges_filename  = '../../data/stanford/facebook/348.edges'
_lu_  = {'fm':[], 'to':[]}
_pos_ = {}
for _edge_ in open(edges_filename, 'rt').read().split('\n'):
    if _edge_ == '': continue
    _split_     = _edge_.split()
    _fm_, _to_  = int(_split_[0]), int(_split_[1])
    _lu_['fm'].append(_fm_), _lu_['to'].append(_to_)
    _pos_[_fm_] = [10.0+random.random()*100.0, 20.0+random.random()*100.0]
    _pos_[_to_] = [10.0+random.random()*100.0, 20.0+random.random()*100.0]
'''
_lu_  = {'fm':'a b c a b c c c c c c c'.split(), 'to':'b c a a0 b0 c0 c1 c2 c3 c4 c5 c6'.split()}
_df_  = pl.DataFrame(_lu_)
_g_   = rt.createNetworkXGraph(_df_, [('fm','to')])
_pos_ = {}
for _node_ in _g_.nodes(): _pos_[_node_] = [random.random(), random.random()]
_iterations_ = None
t0 = time.time()
_pos_ref_ = rt.springLayout   (_g_, copy.deepcopy(_pos_), iterations=_iterations_)
t1 = time.time()
_psl_     = PolarsSpringLayout(_g_, copy.deepcopy(_pos_), iterations=_iterations_)
_pos_     = _psl_.results()
t2 = time.time()
_pos_ref2_ = nx.spring_layout(_g_)
t3 = time.time()
'''
rt.tile([rt.link(_df_, [('fm','to')], _pos_ref_), 
         rt.link(_df_, [('fm','to')], _pos_ref2_), 
         rt.link(_df_, [('fm','to')], _pos_)], spacer=10)
'''

In [ ]:
# Ref Time = 5.1s | Polars Time = 0.8s | M1 Pro @ 16GB
# Ref Time = 5.14s, Polars Time = 0.82s, NX Ref Time = 0.06s | M1 Pro @ 16GB
# Ref Time = 5.15s, Polars Time = 0.79s, NX Ref Time = 0.05s | M1 Pro @ 16GB // removed dx and dy tables from polars impl
# Ref Time = 5.16s, Polars Time = 0.83s, NX Ref Time = 0.05s | M1 Pro @ 16GB // with the static nodes requirement
# 
print(f'Ref Time = {t1-t0:.2f}s, Polars Time = {t2-t1:.2f}s, NX Ref Time = {t3-t2:.2f}s')

In [ ]:
def graphToDataFrame(_g_):
    _lu_ = {'fm':[], 'to':[]}
    for _node_ in _g_.nodes(): 
        for _nbor_ in _g_.neighbors(_node_):
            _lu_['fm'].append(_node_), _lu_['to'].append(_nbor_)
    return pl.DataFrame(_lu_), [('fm','to')]

_patterns_ = LinkNodeGraphPatterns()
_tiles_    = []
for _type_ in _patterns_.types:
    _g_             = _patterns_.createPattern(_type_)
    _df_, _relates_ = graphToDataFrame(_g_)
    _pos1_ = nx.spring_layout(_g_)
    _pos2_ = rt.springLayout(_g_)
    _pos3_ = PolarsSpringLayout(_g_).results()
    _tiles_.append(rt.link(_df_, _relates_, _pos1_))
    _tiles_.append(rt.link(_df_, _relates_, _pos2_))
    _tiles_.append(rt.link(_df_, _relates_, _pos3_))
#rt.table(_tiles_, per_row=3, spacer=10)

In [ ]:
_pos_ = PolarsSpringLayout(_g_).results()
_tiles_ = []
for i in range(10):
    _node_colors_ = {}
    _lms_, _shortests_ = rt.identifyLandmarksInConnectedComponent(_g_, 3)
    for _landmark_ in _lms_: _node_colors_[_landmark_] = 'red'
    _tiles_.append(rt.link(_df_, _relates_, _pos_, node_color=_node_colors_))
rt.table(_tiles_, per_row=5)